In [ ]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine, inspect
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Brandon's code starts here

In [ ]:
# Brandon's code ends here

In [ ]:
# Mauvonte's code starts here

In [ ]:
# Mauvonte's code ends here

In [ ]:
# Scott's code starts here

In [ ]:
# Scott's code ends here